In [1]:
from daft import DataFrame, col, udf

IMAGES_CSV_S3_PATH = "s3://amazon-berkeley-objects/images/metadata/images.csv.gz"
images_df = DataFrame.from_csv(IMAGES_CSV_S3_PATH)

2022-08-29 16:36:52.744 | INFO     | daft.dataframe:<module>:45 - Using PyRunner


In [2]:
images_df

image_id(STRING),height(INTEGER),width(INTEGER),path(STRING)
010-mllS7JL,106,106,14/14fe8812.jpg
01dkn0Gyx0L,122,122,da/daab0cad.jpg
01sUPg0387L,111,111,d2/d2daaae9.jpg
1168jc-5r1L,186,186,3a/3a4e88e6.jpg
11RUV5Fs65L,30,500,d9/d91ab9cf.jpg
11X4pFHqYOL,35,500,20/20098c4d.jpg
11Y+Xpt1lfL,103,196,99/9987a1c8.jpg
11rL64ZLPYL,64,500,89/89a2ff4d.jpg
11xjmNF5TAL,117,88,ee/ee239f0f.jpg
11xkwXwrSXL,80,500,75/75536bf9.jpg


In [3]:
LISTING_JSON_S3_PATH = "s3://amazon-berkeley-objects/listings/metadata/listings_0.json.gz"
listings_df = DataFrame.from_json(LISTING_JSON_S3_PATH)

In [4]:
listings_df

brand(PyObj[object]),bullet_point(PyObj[object]),color(PyObj[object]),item_id(STRING),item_name(PyObj[object]),model_name(PyObj[object]),model_number(PyObj[object]),model_year(PyObj[object]),product_type(PyObj[object]),style(PyObj[object]),main_image_id(STRING),other_image_id(PyObj[object]),item_keywords(PyObj[object]),country(STRING),marketplace(STRING),domain_name(STRING),node(PyObj[object]),item_dimensions(PyObj[object]),item_weight(PyObj[object]),material(PyObj[object]),fabric_type(PyObj[object]),color_code(PyObj[object]),product_description(PyObj[object]),spin_id(STRING),3dmodel_id(STRING),pattern(PyObj[object]),finish_type(PyObj[object]),item_shape(PyObj[object])
"[{'language_tag': 'nl_NL', 'value': 'find.'}]","[{'language_tag': 'nl_NL', 'value': 'Schoen in Loafer- st...","[{'language_tag': 'nl_NL', 'value': 'Veelkleurig Vrouw B...",B06X9STHNG,"[{'language_tag': 'nl_NL', 'value': 'Amazon-merk - vinde...","[{'language_tag': 'nl_NL', 'value': '37753'}]",[{'value': '12-05-04'}],[{'value': 2017}],[{'value': 'SHOES'}],"[{'language_tag': 'nl_NL', 'value': 'Gesloten-teen pompen'}]",81iZlv3bjpL,['91mIRxgziUL' '91eqBkW06wL' 'A1BHZSKNbkL'],"[{'language_tag': 'nl_NL', 'value': 'block heel shoes'} ...",NL,Amazon,amazon.nl,"[{'node_id': 16391787031, 'node_name': '/Categorieën/Dam...",None,None,None,None,None,None,None,None,None,None,None
"[{'language_tag': 'es_MX', 'value': 'AmazonBasics'}]","[{'language_tag': 'es_MX', 'value': 'White Powder Coat F...","[{'language_tag': 'es_MX', 'value': 'White Powder Coat',...",B07P8ML82R,"[{'language_tag': 'es_MX', 'value': '22"" Bottom Mount Dr...",None,[{'value': 'AB5013-R22-10'}],None,[{'value': 'HARDWARE'}],"[{'language_tag': 'es_MX', 'value': '10 pares'}]",619y9YG9cnL,['51Fqps5k9YL' '51lCKFuYuWL'],"[{'language_tag': 'es_MX', 'value': '22'} {'language_ta...",MX,Amazon,amazon.com.mx,"[{'node_id': 9827962011, 'node_name': '/Categ orías/Ferre...","{'height': {'normalized_value': {'unit': 'inches', 'valu...","[{'normalized_value' : {'unit': 'pounds', 'value': 1.45},...",None,None,None,None,None,None,None,None,None
"[{'language_tag': 'en_AE', 'value': 'AmazonBasics'}]","[{'language_tag': 'en_AE', 'value': '3D printer filament...","[{'language_tag': 'en_AE', 'value': 'Translucent Yellow'...",B07H9GMYXS,"[{'language_tag': 'en_AE', 'value': 'AmazonBasics PETG 3...",None,[{'value': 'AMG10528516-10'}],None,[{'value': 'MECHANIC AL_COMPONENTS'}],"[{'language_tag': 'en_AE', 'value': '1-Pack'}]",81NP7qh2L6L,['81A0u5L4VAL' '61xhS6iLrZL'],"[{'language_tag': 'en_AE', 'value': '3d printer filament...",AE,Amazon,amazon.ae,"[{'node_id': 11601270031, 'node_name': '/Categories'}]","{'height': {'normalized_value': {'unit': 'inches', 'valu...","[{'normalized_value' : {'unit': 'pounds', 'value': 2.2}, ...","[{'language_tag': 'en_AE', 'value': 'PETG (Polyethylene ...",None,None,None,None,None,None,None,None
"[{'language_tag': 'en_GB', 'value': 'Stone & Beam'}]",None,"[{'language_tag': 'en_GB', 'value': 'Stone Brown', 'stan...",B07CTPR73M,"[{'language_tag': 'en_GB', 'value': 'Stone & Beam Stone ...",None,[{'value': '25020039-01'}],None,[{'value': 'SOFA'}],None,61Rp4qOih9L,None,"[{'language_tag': 'en_GB', 'value': 'love'} {'language_...",GB,Amazon,amazon.co.uk,"[{'node_id': 2850919031, 'node_name': '/Home & Garden/Ho...",None,"[{'normalized_value' : {'unit': 'pounds', 'value': 0.1}, ...",None,None,None,None,None,None,None,None,None
"[{'language_tag': 'en_AU', 'value': 'The Fix'}]","[{'language_tag': 'en_AU', 'value': 'Embroidered flowers...","[{'language_tag': 'en_AU', 'value': 'Havana Tan', 'stand...",B01MTEI8M6,"[{'language_tag': 'en_AU', 'value': ""The Fix Amazon Bran...",None,[{'value': '880071Havana Tan'}],[{'value': 2017}],[{'value': 'SHOES'}],"[{'language_tag': 'en_AU', 'value': 'French Loafer Slide'}]",714CmIfKIYL,['71C4hQAAs2L' '718uEco1DAL' '71BMHcaG5GL' '7105JsM1HYL'...,"[{'language_tag': 'en_AU', 'value': 'zapatos shoe para d...",AU,Amazon,amazon.com.au,"[{'node_id': 5131142051, 'node_name': '/Categ ories/Wom

In [5]:
listings_df.select(col("bullet_point"), col("product_type"), col("item_name"), col("material"))

bullet_point(PyObj[object]),product_type(PyObj[object]),item_name(PyObj[object]),material(PyObj[object])
"[{'language_tag': 'nl_NL', 'value': 'Schoen in Loafer- st...",[{'value': 'SHOES'}],"[{'language_tag': 'nl_NL', 'value': 'Amazon-merk - vinde...",None
"[{'language_tag': 'es_MX', 'value': 'White Powder Coat F...",[{'value': 'HARDWARE'}],"[{'language_tag': 'es_MX', 'value': '22"" Bottom Mount Dr...",None
"[{'language_tag': 'en_AE', 'value': '3D printer filament...",[{'value': 'MECHANIC AL_COMPONENTS'}],"[{'language_tag': 'en_AE', 'value': 'AmazonBasics PETG 3...","[{'language_tag': 'en_AE', 'value': 'PETG (Polyethylene ..."
None,[{'value': 'SOFA'}],"[{'language_tag': 'en_GB', 'value': 'Stone & Beam Stone ...",None
"[{'language_tag': 'en_AU', 'value': 'Embroidered flowers...",[{'value': 'SHOES'}],"[{'language_tag': 'en_AU', 'value': ""The Fix Amazon Bran...",None
"[{'language_tag': 'en_IN', 'value': 'Snug fit for Mi Red...",[{'value': 'CELLULAR _PHONE_CASE'}],"[{'language_tag': 'en_IN', 'value': 'Amazon Brand - Soli...","[{'language_tag': 'en_IN', 'value': 'Plastic'}]"
"[{'language_tag': 'en_IN', 'value': 'Snug fit for Xiaomi...",[{'value': 'CELLULAR _PHONE_CASE'}],"[{'language_tag': 'en_IN', 'value': 'Amazon Brand - Soli...",None
"[{'language_tag': 'en_US', 'value': 'Lead-free glass: ma...",[{'value': 'DRINKING_CUP'}],"[{'language_tag': 'en_US', 'value': 'AmazonBasics Serene...",None
"[{'language_tag': 'en_IN', 'value': 'Snug fit for Vivo Y...",[{'value': 'CELLULAR _PHONE_CASE'}],"[{'language_tag': 'en_IN', 'value': 'Amazon Brand - Soli...","[{'language_tag': 'en_IN', 'value': 'Plastic'}]"
"[{'language_tag': 'en_GB', 'value': 'Product Detail: Pac...",[{'value': 'FURNITURE_COVER'}],"[{'language_tag': 'en_GB', 'value': 'UMI by Amazon 6 PCS...","[{'language_tag': 'en_GB', 'value': 'Velvet'}]"


In [6]:
from daft.expressions import udf
import pandas as pd

@udf(return_type=dict)
def get_first(lists):
    return [l[0] if len(l) > 0 else None for l in lists]

@udf(return_type=str)
def get_first_en_or_none(lists):
    lists = [[item["value"] for item in l if item["language_tag"].startswith("en")] if l  is not None else None for l in lists]
    lists = [en_items[0] if en_items else None for en_items in lists]
    return pd.Series(lists)

@udf(return_type=str)
def get_en_concat(lists):
    lists = [[item["value"] for item in l if item["language_tag"].startswith("en")] if l is not None else None for l in lists]
    lists = [" ".join(en_items) if en_items else None for en_items in lists]
    return lists

@udf(return_type=bool)
def is_not_null(bullet_points):
    return bullet_points != None

@udf(return_type=str)
def cast_str(l):
    return pd.Series(l)

processed_listings = listings_df.with_column(
    "product_type_parsed", cast_str(get_first(col("product_type")).as_py(dict)["value"])
).where(
    col("product_type_parsed") == "SHOES"
).select(
    get_en_concat(col("bullet_point")).alias("details"),
    col("product_type_parsed"),
    get_first_en_or_none(col("item_name")),
    col("main_image_id")
).where(
    is_not_null(col("details"))
)

In [7]:
processed_listings

details(STRING),product_type_parsed(STRING),item_name(STRING),main_image_id(STRING)
Embroidered flowers bloom against understated tan suede ...,SHOES,The Fix Amazon Brand Women's French Floral Embroidery Sl...,714CmIfKIYL
Outer Material: PU Closure Type: Slip On Toe Style: Roun...,SHOES,Stanton Men's Brown Loafers-8 UK (42 EU) (9 US) (FK/YT18...,71kE0ihplPL
Bright metallic tassels gleam against an earthy suede up...,SHOES,Amazon Brand - The Fix Women's Foley Tassel Slide Sandal...,71-eMu0gN+L
Moccasin-style loafer featuring suede upper and wraparou...,SHOES,Amazon Brand - 206 Collective Men's Pike Driving Slip- on...,61uONZVpAEL
Outer Material: Pu Product Type: Men's Formal Shoes Shoe...,SHOES,Amazon Brand - Symbol Men's Tan PU Formal Shoes - 9 UK (...,814RXC4DgkL
Classic and versatile design for everyday use and an exc...,SHOES,"Amazon Essentials Women's Manny, Brown Tan, 38.5 EU",61fH9aTfMIL
Outer Material: Leather Product Type: Men's Formal Shoes...,SHOES,Amazon Brand - Symbol Men's Leather Formal Slip-on shoes...,71cyz3P4DKL
Outer Material: Mesh Closure Type: Lace- Up Toe Style: Ro...,SHOES,Bourge Men's D.Grey and Sea Green Running Shoes-6 UK (40...,81L8EV8ocAL
Desert-boot stitching Soft natural leather,SHOES,"find. Suede Derbys, Black), 7 UK",71w+9SNWhbL
Outer Material: Leather Closure Type: Lace-Up Heel type:...,SHOES,Burwood Men's Black Leather Formal Shoes-6 UK (40 EU) (B...,715Q4uySlWL


In [8]:
joined_df = processed_listings.join(images_df, left_on=col("main_image_id"), right_on=col("image_id"))

In [9]:
joined_df

details(STRING),product_type_parsed(STRING),item_name(STRING),main_image_id(STRING),height(INTEGER),width(INTEGER),path(STRING)
Embroidered flowers bloom against understated tan suede ...,SHOES,The Fix Amazon Brand Women's French Floral Embroidery Sl...,714CmIfKIYL,868,1779,2b/2b1c2516.jpg
Outer Material: PU Closure Type: Slip On Toe Style: Roun...,SHOES,Stanton Men's Brown Loafers-8 UK (42 EU) (9 US) (FK/YT18...,71kE0ihplPL,2000,2000,cd/cd870277.jpg
Bright metallic tassels gleam against an earthy suede up...,SHOES,Amazon Brand - The Fix Women's Foley Tassel Slide Sandal...,71-eMu0gN+L,704,1770,2c/2c0416de.jpg
Moccasin-style loafer featuring suede upper and wraparou...,SHOES,Amazon Brand - 206 Collective Men's Pike Driving Slip- on...,61uONZVpAEL,802,2037,14/14d129a0.jpg
Outer Material: Pu Product Type: Men's Formal Shoes Shoe...,SHOES,Amazon Brand - Symbol Men's Tan PU Formal Shoes - 9 UK (...,814RXC4DgkL,1324,2517,f4/f4f31eec.jpg
Classic and versatile design for everyday use and an exc...,SHOES,"Amazon Essentials Women's Manny, Brown Tan, 38.5 EU",61fH9aTfMIL,891,2080,05/051b1105.jpg
Outer Material: Leather Product Type: Men's Formal Shoes...,SHOES,Amazon Brand - Symbol Men's Leather Formal Slip-on shoes...,71cyz3P4DKL,2560,2560,b6/b630bf31.jpg
Outer Material: Mesh Closure Type: Lace- Up Toe Style: Ro...,SHOES,Bourge Men's D.Grey and Sea Green Running Shoes-6 UK (40...,81L8EV8ocAL,2000,2000,14/14db355d.jpg
Desert-boot stitching Soft natural leather,SHOES,"find. Suede Derbys, Black), 7 UK",71w+9SNWhbL,2560,1969,03/03b9ac4f.jpg
Outer Material: Leather Closure Type: Lace-Up Heel type:...,SHOES,Burwood Men's Black Leather Formal Shoes-6 UK (40 EU) (B...,715Q4uySlWL,2000,2000,12/120249ec.jpg


In [10]:
import concurrent.futures
import threading
import PIL.Image
import boto3
import io


@udf(return_type=PIL.Image.Image)
def download_batch(batch):
    def download_single(obj: str) -> bytes:
        local = threading.local()
        if "boto_session" not in local.__dict__:
            local.boto_session = boto3.session.Session()
        s3 = local.boto_session.client('s3')
        bucket, key = obj.replace("s3://", "").split("/", maxsplit=1)
        response = s3.get_object(Bucket=bucket, Key=key)
        body = response["Body"]
        contents = body.read()
        body.close()
        return contents
    
    with concurrent.futures.ThreadPoolExecutor() as executor : 
        byte_contents = [res for res in executor.map(download_single, batch)]
        images = []
        for payload in byte_contents:
            with io.BytesIO(payload) as f:
                images.append(PIL.Image.open(f).convert("RGB"))
        return images
    
@udf(return_type=str)
def full_url(paths):
    return pd.Series(["s3://amazon-berkeley-objects/images/small/" + path for path in paths])

In [11]:
joined_df = joined_df.with_column("s3_url", full_url(col("path")))
joined_df = joined_df.with_column("image", download_batch(col("s3_url")))

In [12]:
joined_df

details(STRING),product_type_parsed(STRING),item_name(STRING),main_image_id(STRING),height(INTEGER),width(INTEGER),path(STRING),s3_url(STRING),image(PyObj[Image])
Embroidered flowers bloom against understated tan suede ...,SHOES,The Fix Amazon Brand Women's French Floral Embroidery Sl...,714CmIfKIYL,868,1779,2b/2b1c2516.jpg,s3://amazon- berkeley-objects/ima ges/small/2b/2b1c251 6.jpg,"<img src=""data:image /jpeg;base64, /9j/4A AQSkZJRgABAQAAAQABAA D/2wBDAAgGBgcGBQgHBw cJCQgKDBQNDAsLDBkSEw 8UHRofHh0aHBwgJC4nIC IsIxwcKDcpLDAxNDQ0Hy c5PTgyPC4zNDL/2wBDAQ kJCQwLDBgNDRgyIRwhMj IyMjIyMjIyMjIyMjIyMj IyMjIyMjIyMjIyMjIyMj IyMjIyMjIyMjIyMjIyMj IyMjL/wAARCAA/AIADAS IAAhEBAxEB/8QAHwAAAQ UBAQEBAQEAAAAAAAAAAA ECAwQFBgcICQoL/8QAtR AAAgEDAwIEAwUFBAQAAA F9AQIDAAQRBRIhMUEGE1 FhByJxFDKBkaEII0KxwR VS0fAkM2JyggkKFhcYGR olJicoKSo0NTY3ODk6Q0 RFRkdISUpTVFVWV1hZWm NkZWZnaGlqc3R1dnd4eX qDhIWGh4iJipKTlJWWl5 iZmqKjpKWmp6ipqrKztL W2t7i5usLDxMXGx8jJyt LT1NXW19jZ2uHi4+Tl5u fo6erx8vP09fb3+Pn6/8 QAHwEAAwEBAQEBAQEBAQ AAAAAAAAECAwQFBgcICQ oL/8QAtREAAgECBAQDBA cFBAQAAQJ3AAECAxEEBS ExBhJBUQdhcRMiMoEIFE KRobHBCSMzUvAVYnLRCh YkNOEl8RcYGRomJygpKj U2Nzg5OkNERUZHSElKU1 RVVldYWVpjZGVmZ2hpan N0dXZ3eHl6goOEhYaHiI mKkpOUlZaXmJmaoqOkpa anqKmqsrO0tba3uLm6ws PExcbHyMnK0tPU1dbX2N na4uPk5ebn6Onq8vP09f b3+Pn6/9oADAMBAAIRAx EAPwD3+iiigAooooAKKK KACjpVDU9XtdKh3ztlj9 2Neprg/EGv3utoLaG6ks Lb/losJ+aT2Ldh7ColUj HRsuNOUtjd8TeKvskL22 msJLnHLKRx7AnjPvXLjU r/AAssOp3Ku3OHcnmsGC GW1d1mneZSchnO7nt+VZ 9tf6m93cWzfZlKNjBDDA /vgdxiued53tLY2oQlKV kvwPStH8ayRSi21gDB4E 6j+Y/qK7hHWRA6EMpGQQ cgivCrS5a7E0Nyy+bFIU JX9G/GvRfAGpSz2dxYys WFuQYyeyntVUqkubkkFa kkro7Kiiiuk5gooooAKK KKACiioZ7qC2QvPMkajk l2AoAmrmNe8ZafpjPbQ3 MD3Q4YFxhPr7+1ZOvfEe wthJbWRaR+VMoU8fQf1r iYbpJ0YiMNGORhRg/Wsa k3b3TroYSrPXl0NG81M3 kpneUyO3O7OaoPKSc5NV byC1jtpLqGCRZF6C1GCx +nT8xUAu57aBJryLETDJ kXGY/98Dp9R+lcTi2rm9 lF8r3NEyBl5X8az5ra4S ++2W02GMYRo9oYMAcjGe lWRIroCrhkbkEHINKjAH JFKMnF3Q/NOxWs7Iw3kl 08zSyyLtPyhRjPHTvXrf g3RX0vTWnnTbcXOGK91U dB9a4nw7e6PaaibrUFcm NcxgLuG7PU+/pXWyfEHT VJEcE7gdzgV005K/PN6n PW5paJHXUVytv490uZsS RzxD+8QCP0roLPULTUIv MtJ0lXvtPI+o7V0xnGWz OZxa3RaoooqiQ6Vxuo+P raK4lttPjinkjJUu8oC5 HsOT+lZ/jnxU9vctpFo+ zaoM7g8nP8NeeSvb3Dfv YY3x3Yc1zVazTtE6KVG6 uzt5vEusXUf7y6EYbgrF 8oX8uT+dc5d28swJeSW4 Y9d0pA9qxhaWZ+7GUH+z Iw/rUqW9uhO0y/9/m/xr n9pK9ztoS9jeyRELO/WU stsqAdi4x/OtCOVFXFxc xJjsHBqv5VsT/qw5B/jJ b+dSRbE+7Eij/ZUCk5vo dX12paySJZr6y8ny0dpC O6IWP51WSeXyikNm7Fjy 0uFB/Cr6TLjAIAxUbuuC c0c7tynnulF1Pav4jD8i fTZfNBjS3IPmQQxkge45 61fSYyRqyupBGdyngj2p 7kEkg5PcGs1j9iuPS3kP p9xv8AA0ty9jUEh6dajZ thJ5we1RmZUjbB5zUDSs U3envQkDZaSfGeufrVuy 1i5066W4tp2jkX0PUeh9 RWG8hUbicD61Tm1OJWwX BPoKtJohtdT3/RvF1jqe nJcNmOT7sidcN/hWrFq1 jNKsSXKeY/CoTgmvnC11 qaGNlildA33gM811XgK5 a88Z2CuWbBZvmPohNbRr SbsYSpJK5S+Ipm0/xhfe ZnErCRG7FSB/8Aqrl49S JOMj86+nrzT7PUIfKvLW G4j/uyoGH61y198LfCd8 xY6e0DHvBIVq5Ubu4o1r Kx4iL85znNWI77ecEgV6 Tc/BTSmybTVb2D0DgOP6 VlzfBfUEz9m1uBx6Swlf 5ZqHRZarI5CO6ABwetO+ 2EDAPFdA/wl8SxE7Lqyk +jEfzqpL8N/FUP/LCB8e kyj+tT7J9ivaruZYvOwN SLdE81O3gnxMnBsYj9Lh f8ali8EeKH4XTkP/bwn+ NT7N9h+0RW88Oeahuts8 TI3KsMVv2/w88UyH5rG3 j93uVx+ma2rD4V6g7A6j qUEKd1t1Lt+ZwP501Sl2 E6sTzqyaS5jjRVZ5M7Tj uQcdK9D0L4bXd9brNqcz WkbciNVBkP1zwK7jQfB+ j+HUBs7fdN3nlO5/z7fh W/W0aK3ZjKs3ojyfUvgn HdsWg8Q3SHsskQYfoRXN zfBDXba6Vre+s7mPOMkl GHuQR/WvfKK0dOJCqSPG rL4MagSDearbxjuI0Zz+ uK7Xw18PNP8OXqXy3U9x coCFLAKoyMHgf412FFCp xWwOpJn//Z"" alt="""" />"
Outer Material: PU Closure Type: Slip On Toe Style: Roun...,SHOES,Stanton Men's Brown Loafers-8 UK (42 EU) (9 US) (FK/YT18...,71kE0ihplPL,2000,2000,cd/cd870277.jpg,s3://amazon- berkeley-objects/ima ges/small/cd/cd87027 7.jpg,"<img src=""data:image /jpeg;base64, /9j/4A AQSkZJRgABAQAAAQABAA D/2wBDAAgGBgcGBQgHBw cJCQgKDBQNDAsLDBkSEw 8UHRofHh0aHBwgJC4nIC IsIxwcKDcpLDAxNDQ0Hy c5PTgyPC4zNDL/2wBDAQ kJCQwLDBgNDRgyIRwhMj IyMjIyMjIyMjIyMjIyMj IyMjIyMjIyMjIyMjIyMj IyMjIyMjIyMjIyMjIyMj IyMjL/wAARCACAAIADAS IAAhEBAxEB/8QAHwAAAQ UBAQEBAQEAAAAAAAAAAA ECAwQFBgcICQoL/8QAtR AAAgEDAwIEAwUFBAQAAA F9AQIDAAQRBRIhMUEGE1 FhByJxFDKBkaEII0KxwR VS0fAkM2JyggkKFhcYG

In [13]:
import torch
from min_dalle import MinDalle

# TODO(jay): We should provide a UDF API to do expensive initializations once only
model = MinDalle(
    models_root='/tmp/pretrained',
    dtype=torch.float32,
    device='cpu',
    is_mega=False, 
    is_reusable=True
)

@udf(return_type=PIL.Image.Image)
def generate_image_from_text(details):
    return [
        model.generate_image(
            text=detail,
            seed=-1,
            grid_size=1,
            is_seamless=False,
            temperature=1,
            top_k=256,
            supercondition_factor=32,
            is_verbose=False
        ) for detail in details
    ]

joined_df = joined_df.with_column("generated_image", generate_image_from_text(col("details")))

using device cpu
downloading tokenizer params
intializing TextTokenizer
downloading encoder params
initializing DalleBartEncoder
downloading decoder params
initializing DalleBartDecoder
downloading detokenizer params
initializing VQGanDetokenizer


In [ ]:
joined_df.limit(1)